In [1]:
import pandas as pd
from PIL import Image
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tqdm import tqdm
from sklearn.model_selection import train_test_split

### Caricamento delle immagini in input in formato array numpy

In [2]:
# IMAGE_PATH_FOLDER = './Dati Sensori/Dataset Sensori Lavorato/1_M/1_M_1'
IMAGE_PATH_FOLDER = '../../Nuovi Dati/DatasetDistorti'

# Specificare solamente una tipologia di speckle
DISTORSIONI = ['DatasetBlackStains-t']
GRADI_DISTORSIONE = 5

HEIGHT = 512
WIDTH = 512

TARGET_SIZE = (WIDTH, HEIGHT)
CHANNELS = 1 # Scala di grigi

In [3]:
def get_sotto_cartelle(path):
    return [x[1] for x in os.walk(path)][0]

def get_immagini_cartella(path_cartella):
    immagini_cartella = []
    cartelle_soggetti = get_sotto_cartelle(path_cartella)
    for soggetto in tqdm(cartelle_soggetti):
        path_soggetto = f'{path_cartella}/{soggetto}'
        cartelle_acquisizioni = get_sotto_cartelle(path_soggetto)
        for acquisizione in cartelle_acquisizioni:
            path_acquisizione = f'{path_soggetto}/{acquisizione}'
            for filename in os.listdir(path_acquisizione):
                if (filename.endswith(".jpg")):    
                    path_file = f'{path_acquisizione}/{filename}'
                    immagine = get_immagine_da_path(path_file)
                    immagini_cartella.append(immagine)
    return immagini_cartella
                    
def get_immagine_da_path(path_file):
    image = Image.open(path_file).convert('L')
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    return normalized_image_array


In [4]:
image_list = []
label_list = []
LISTA_DISTORSIONI = [0, 2, 3, 4]

for distorsione in DISTORSIONI:
    path_distorsione = f'{IMAGE_PATH_FOLDER}/{distorsione}'
    for grado_distorsione in LISTA_DISTORSIONI:
        label = 100 - float(25 * (grado_distorsione))
        path_cartella_grado = f'{path_distorsione}{grado_distorsione}'
        immagini_grado = get_immagini_cartella(path_cartella_grado)
        image_list.extend(immagini_grado)
        label_to_append = [label for x in range(len(immagini_grado))] 
        label_list.extend(label_to_append)
        

100%|██████████| 719/719 [00:02<00:00, 244.92it/s]


In [5]:
train_images = np.array(image_list)
train_weights = np.array(label_list)

In [6]:
X = train_images
y = train_weights

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(HEIGHT, WIDTH, CHANNELS)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  
])

Metal device set to: Apple M1 Pro


2023-07-25 14:45:33.931032: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-25 14:45:33.933263: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
model.compile(optimizer='adam', loss='mae', metrics = ['mae'])

In [10]:
model.fit(x=X_train, y=y_train, batch_size=1, epochs=12)

Epoch 1/12


2023-07-25 14:45:56.145345: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-07-25 14:45:56.626448: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2300/2300 [==============================] - 249s 105ms/step - loss: 14.0037 - mae: 14.0037
Epoch 2/12
2300/2300 [==============================] - 243s 105ms/step - loss: 6.5208 - mae: 6.5208
Epoch 3/12
2300/2300 [==============================] - 243s 106ms/step - loss: 4.7388 - mae: 4.7388
Epoch 4/12
2300/2300 [==============================] - 242s 105ms/step - loss: 3.9222 - mae: 3.9222
Epoch 5/12
2300/2300 [==============================] - 242s 105ms/step - loss: 3.7923 - mae: 3.7923
Epoch 6/12
2300/2300 [==============================] - 242s 105ms/step - loss: 3.6884 - mae: 3.6884
Epoch 7/12
2300/2300 [==============================] - 242s 105ms/step - loss: 3.2622 - mae: 3.2622
Epoch 8/12
2300/2300 [==============================] - 242s 105ms/step - loss: 3.2335 - mae: 3.2335
Epoch 9/12
2300/2300 [==============================] - 242s 105ms/step - loss: 3.1681 - mae: 3.1681
Epoch 10/12
2300/2300 [==============================] - 242s 105ms/step - loss: 3.0698 - mae: 3.069

In [11]:
predictions = model.predict(X_test)

 1/18 [>.............................] - ETA: 3s

2023-07-25 15:34:35.483654: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/18 [==============================] - 4s 243ms/step


In [12]:
predictions_list = [i[0] for i in predictions] 

In [13]:
clamp_pred = [min(100,i) for i in predictions_list]
clamp_pred = [max(0,i) for i in clamp_pred]

In [14]:
max(abs(clamp_pred  - y_test))

75.97044563293457

In [17]:
def avg(lst):
    return sum(lst) / len(lst)

avg(abs(clamp_pred - y_test))

1.8492218669917848

In [24]:
clamp_pred[450:460]

[0, 49.783627, 100, 0, 23.88472, 27.18199, 42.88512, 49.5971, 25.042871, 0]

In [25]:
y_test[450:460]

array([  0.,  50., 100.,   0.,  25., 100.,  50.,  50.,  25.,   0.])

In [20]:
a = abs(clamp_pred - y_test)


In [21]:
for i, el in enumerate(list(a)):
    print(f'{i}: {el}')

0: 0.0
1: 1.235107421875
2: 0.5888748168945312
3: 0.0
4: 1.2427330017089844
5: 7.251091003417969
6: 0.0
7: 0.0
8: 0.049472808837890625
9: 0.0
10: 1.6501121520996094
11: 2.167062759399414
12: 0.0122222900390625
13: 0.0
14: 0.0
15: 1.6936511993408203
16: 0.0
17: 1.1250419616699219
18: 1.400156021118164
19: 0.6808547973632812
20: 0.0
21: 0.031314849853515625
22: 0.0
23: 0.9846820831298828
24: 0.0
25: 0.0
26: 0.43979644775390625
27: 0.4161338806152344
28: 0.1709747314453125
29: 2.521240234375
30: 2.0830230712890625
31: 18.514846801757812
32: 5.407234191894531
33: 0.0
34: 0.0
35: 0.0
36: 0.012241363525390625
37: 0.0
38: 0.3206214904785156
39: 0.0
40: 0.7388114929199219
41: 0.030902862548828125
42: 0.0
43: 0.0
44: 0.0
45: 47.735252380371094
46: 2.9652671813964844
47: 0.0
48: 0.4823036193847656
49: 0.0
50: 2.1639022827148438
51: 0.05693817138671875
52: 0.0
53: 0.16806411743164062
54: 0.0
55: 0.7943534851074219
56: 25.0
57: 0.3879356384277344
58: 1.7381973266601562
59: 0.020519256591796875
60:

In [16]:
model.save('../../Modelli-v2/modello_pesi_inizializzati_black_stains')

INFO:tensorflow:Assets written to: ../../Modelli-v2/modello_pesi_inizializzati_black_stains/assets


INFO:tensorflow:Assets written to: ../../Modelli-v2/modello_pesi_inizializzati_black_stains/assets
